In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import os.path
from datetime import datetime
import sqlalchemy as sa
#from sqlalchemy.sql import text

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
s="producto50"
n="producto50_DefuncionesDEIS_sospechososPorComuna_std"
sospechosos=pd.read_sql_table(n, con=cnx,schema=s).drop(['index','reverse_idx','MAX_DATE'],axis=1)

In [5]:
s="producto50"
n="producto50_DefuncionesDEIS_confirmadosPorComuna_std"
confirmados=pd.read_sql_table(n, con=cnx,schema=s).drop(['index','reverse_idx','MAX_DATE'],axis=1)

In [6]:
confirmados['Defunciones'].sum()

38234.0

In [7]:
sospechosos['Defunciones'].sum()

10910.0

In [8]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS does not exist, creating it


In [9]:
DEIS=pd.concat([confirmados,sospechosos],keys=['Confirmados','Sospechosos']).reset_index().drop('level_1',axis=1).rename(columns={'level_0':'PCR+'})

In [10]:
DEIS.to_sql('deis', schema='DEIS',con=cnx,if_exists='replace')

In [11]:
DEIS

,PCR+,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Fecha,Defunciones,Region_Metropolitana
0,Confirmados,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-03-16,0.0,Resto de Chile
1,Confirmados,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-03-16,0.0,Resto de Chile
2,Confirmados,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-03-16,0.0,Resto de Chile
3,Confirmados,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-03-16,0.0,Resto de Chile
4,Confirmados,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,2020-03-16,NaN,Resto de Chile
...,...,...,...,...,...,...,...,...,...
448151,Sospechosos,Magallanes y la Antartica,12,Rio Verde,12103.0,211.0,2021-11-24,0.0,Resto de Chile
448152,Sospechosos,Magallanes y la Antartica,12,San Gregorio,12104.0,681.0,2021-11-24,0.0,Resto de Chile
448153,Sospechosos,Magallanes y la Antartica,12,Timaukel,12303.0,282.0,2021-11-24,0.0,Resto de Chile
448154,Sospechosos,Magallanes y la Antartica,12,Torres del Paine,12402.0,1021.0,2021-11-24,0.0,Resto de Chile


In [12]:
DEIS.columns

Index(['PCR+', 'Region', 'Codigo region', 'Comuna', 'Codigo comuna',
       'Poblacion', 'Fecha', 'Defunciones', 'Region_Metropolitana'],
      dtype='object')

In [13]:
DEIS['Defunciones'].sum()

49144.0

In [14]:
DEIS['PCR+']

0         Confirmados
1         Confirmados
2         Confirmados
3         Confirmados
4         Confirmados
             ...     
448151    Sospechosos
448152    Sospechosos
448153    Sospechosos
448154    Sospechosos
448155    Sospechosos
Name: PCR+, Length: 448156, dtype: object

In [15]:
DEIS.groupby(['Fecha','PCR+'])['Defunciones'].sum().reset_index()

,Fecha,PCR+,Defunciones
0,2020-03-16,Confirmados,0.0
1,2020-03-16,Sospechosos,1.0
2,2020-03-17,Confirmados,0.0
3,2020-03-17,Sospechosos,0.0
4,2020-03-18,Confirmados,0.0
...,...,...,...
1233,2021-11-22,Sospechosos,2.0
1234,2021-11-23,Confirmados,23.0
1235,2021-11-23,Sospechosos,2.0
1236,2021-11-24,Confirmados,12.0


In [16]:
DEIS.melt(id_vars=['Fecha','PCR+'])

,Fecha,PCR+,variable,value
0,2020-03-16,Confirmados,Region,Arica y Parinacota
1,2020-03-16,Confirmados,Region,Arica y Parinacota
2,2020-03-16,Confirmados,Region,Arica y Parinacota
3,2020-03-16,Confirmados,Region,Arica y Parinacota
4,2020-03-16,Confirmados,Region,Arica y Parinacota
...,...,...,...,...
3137087,2021-11-24,Sospechosos,Region_Metropolitana,Resto de Chile
3137088,2021-11-24,Sospechosos,Region_Metropolitana,Resto de Chile
3137089,2021-11-24,Sospechosos,Region_Metropolitana,Resto de Chile
3137090,2021-11-24,Sospechosos,Region_Metropolitana,Resto de Chile


In [17]:
DEIS.pivot(index='Fecha',columns=['PCR+'],values=['Defunciones'])

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
#fallecidos nuevos regionales (DEIS)
fr=DEIS.groupby(['FECHA_DEF','Metropolitana'])['FECHA_STR'].count()
fr=fr.reset_index()
fr.columns=['Fecha','Region Metropolitana','Fallecidos Nuevos']
fr=fr.pivot(index='Fecha',columns=['Region Metropolitana'])
fr=fr.T.reset_index().T
fr.columns=fr.iloc[0].astype(str)+' '+fr.iloc[1].astype(str)+' (DEIS)'
fr=fr.iloc[3:].reset_index()
#fallecidos regional acumulado (deis)
fr=fr.fillna(0)
fra=fr[fr.columns.to_list()[1:]].cumsum()
fra.columns=[x.replace('Nuevos','Acumulados') for x in fra.columns]
#concatenar
fr=pd.concat([fr,fra],axis=1)

In [18]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_DEIS.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/DEIS.ipynb')

0